# Отчет о создании и обучении на датасете MNIST модели CNN (гипотеза 6, реализация 4)










## Содержание

- Список терминов, специфичных для темы отчета
- Введение
- 1 Создание и обучение модели
 - 1.1 Загрузка датасета
 - 1.2 Создание модели
 - 1.3 Обучение модели
 - 1.4 Сохранение модели на диск
- 2 Рекомендации по дальнейшему улучшению/развитию модели
- Ссылки

## Список терминов, специфичных для темы отчета

См. раздел "Машинное обучение" документа "Словарь терминов, специфичных для выполняемых работ и изучаемых материалов в период с 25 сентября 2020 г. по 25 ноября 2020 г.".


## Введение

В настоящем отчете приведен код, соответствующий созданию и обучению модели CNN на датасете MNIST, а также рекомендации по дальнейшему улучшению/развитию модели. Используемый язык программирования – Python. Для создания и обучения модели используются библиотеки MXNet и Gluon.

## 1 Создание и обучение модели

Импортирование необходимых модулей и функций:

In [1]:
import mxnet as mx
from mxnet.optimizer import Adam
from mxnet.gluon import nn
from mxnet import autograd as ag
from mxnet import gluon

Установка вычислительного контекста (для использования при обучении видеокарты, если это возможно):

In [2]:
ctx = [mx.gpu() if mx.test_utils.list_gpus() else mx.cpu()]
print('context:', ctx)

context: [gpu(0)]


### 1.1 Загрузка датасета

Загрузка датасета и установка размера батча равным 200:

In [3]:
mnist = mx.test_utils.get_mnist()

batch_size = 200
train_data = mx.io.NDArrayIter(mnist['train_data'], mnist['train_label'], batch_size, shuffle=True)
val_data = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)

### 1.2 Создание модели

Создание модели, состоящей из 6 слоев:
1. Сверточный слой с параметрами:
	- Количество каналов: 10
	- Размер ядра свертки: 5х5
	- Функция активации: гиперболический тангенс
2. Слой пулинга с параметрами:
	- Размер ядра пулинга: 2х2
	- Шаг: 2
3. Сверточный слой
	- Количество каналов: 25
	- Размер ядра свертки: 5х5
	- Функция активации: гиперболический тангенс
4. Слой пулинга с параметрами:
	- Размер ядра пулинга: 2х2
	- Шаг: 2
5. Полносвязный слой с параметрами:
	- Количество нейронов: 20
	- Функция активации: гиперболический тангенс
6. Полносвязный слой с параметрами:
	- Количество нейронов: 10
	- Функция активации: гиперболический тангенс

In [4]:
net = nn.HybridSequential()
with net.name_scope():
    net.add(nn.Conv2D(10, kernel_size=5, activation='tanh'))
    net.add(nn.MaxPool2D(pool_size=2, strides=2))
    net.add(nn.Conv2D(25, kernel_size=5, activation='tanh'))
    net.add(nn.MaxPool2D(pool_size=2, strides=2))
    net.add(nn.Dense(20, activation='tanh'))
    net.add(nn.Dense(10, activation='tanh'))
net.hybridize()

### 1.3 Обучение модели

Для инициализации сети используется метод Завьера (Xavier). В качестве функции потерь используется SCE (Softmax Cross-Entropy loss). В качестве оптимизатора используется градиентный спуск с инерцией. Темп обучения (learning rate) устанавливается равным 0.6. Обучение производится в течение **300** эпох.

In [5]:
%%time

def getValidationAcc():
    # Use Accuracy as the evaluation metric.
    metric = mx.metric.Accuracy()
    # Reset the validation data iterator.
    val_data.reset()
    # Loop over the validation data iterator.
    for batch in val_data:
        # Splits validation data into multiple slices along batch_axis
        # and copy each slice into a context.
        data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
        # Splits validation label into multiple slices along batch_axis
        # and copy each slice into a context.
        label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
        outputs = []
        for x in data:
            outputs.append(net(x))
        # Updates internal evaluation
        metric.update(label, outputs)
    return metric.get()[1]

validationAccToTrain = 0.99
epochsToTrain = 300
#               ^^--- 40 -> 300

def train(net):
    # Use Accuracy as the evaluation metric.
    metric = mx.metric.Accuracy()
    # Initialize the parameters with Xavier initializer
    net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)
    # Use cross entropy loss
    softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
    # Use Adam optimizer
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.6})
    validationAcc = None
    maxValidationAcc = 0

    for i in range(epochsToTrain):
        # Reset the train data iterator.
        train_data.reset()
        # Loop over the train data iterator.
        for batch in train_data:
            # Splits train data into multiple slices along batch_axis
            # and copy each slice into a context.
            data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
            # Splits train labels into multiple slices along batch_axis
            # and copy each slice into a context.
            label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
            outputs = []
            # Inside training scope
            with ag.record():
                for x, y in zip(data, label):
                    z = net(x)
                    # Computes softmax cross entropy loss.
                    loss = softmax_cross_entropy(z, y)
                    # Backpropogate the error for one iteration.
                    loss.backward()
                    outputs.append(z)
            # Updates internal evaluation
            metric.update(label, outputs)
            # Make one step of parameter update. Trainer needs to know the
            # batch size of data to normalize the gradient by 1/batch_size.
            trainer.step(batch.data[0].shape[0])
        # Gets the evaluation result.
        name, acc = metric.get()
        # Reset evaluation result to initial state.
        metric.reset()
        validationAcc = getValidationAcc()
        epoch = i + 1
        print('epoch', ('{:2.0f}:').format(epoch), '\t', "{:.2f}%".format(acc * 100), '- train set accuracy'
              '\n\t\t', "{:.2f}%".format(validationAcc * 100), '- validation set accuracy')
        if validationAcc > maxValidationAcc:
            maxValidationAcc = validationAcc
            if validationAcc >= validationAccToTrain:
                net.export('exported/300/mnist', epoch=epoch)
                print('saved epoch', epoch)
        print()
    print('Best result: {:.2f}%'.format(maxValidationAcc * 100))

train(net)

epoch  1: 	 90.41% - train set accuracy
		 97.17% - validation set accuracy

epoch  2: 	 97.30% - train set accuracy
		 98.05% - validation set accuracy

epoch  3: 	 98.02% - train set accuracy
		 98.16% - validation set accuracy

epoch  4: 	 98.38% - train set accuracy
		 98.29% - validation set accuracy

epoch  5: 	 98.57% - train set accuracy
		 98.54% - validation set accuracy

epoch  6: 	 98.74% - train set accuracy
		 98.57% - validation set accuracy

epoch  7: 	 98.95% - train set accuracy
		 98.61% - validation set accuracy

epoch  8: 	 99.03% - train set accuracy
		 98.61% - validation set accuracy

epoch  9: 	 99.14% - train set accuracy
		 98.62% - validation set accuracy

epoch 10: 	 99.22% - train set accuracy
		 98.74% - validation set accuracy

epoch 11: 	 99.26% - train set accuracy
		 98.70% - validation set accuracy

epoch 12: 	 99.33% - train set accuracy
		 98.79% - validation set accuracy

epoch 13: 	 99.35% - train set accuracy
		 98.70% - validation set accuracy


Максимальная достигнутая точность на проверочной части датасета составляет **99.02%**

### 1.4 Сохранение модели на диск

Обучение модели выполняется посредством функции `train` (см. раздел 1.3). Функция `train` использует глобальные переменные `validationAccToTrain` и `epochsToTrain`. Значение глобальной переменной `validationAccToTrain` соответствует точности (accuracy) на проверочной части датасета, при достижении которой и одновременно максимальной точности в ходе обучения, необходимо сохранить архитектуру и параметры модели на диск. Сохранение модели на диск выполняется посредством функции `net.export`, при этом создаются/перезаписываются два файла: **mnist-symbol.json**, содержащий информацию, определяющую архитектуру модели, и **mnist-<номер эпохи>.params**, содержащий параметры модели, полученные в ходе обучения и соответствующие эпохе <номер эпохи>. Значение глобальной переменной `epochsToTrain` соответствует количеству эпох, в течение которых необходимо проводить обучение.

## Ссылки

1. Официальный сайт mxnet: https://mxnet.apache.org/